In [37]:
import icalendar
from datetime import datetime, timedelta

In [35]:
# Obtener la fecha actual
fecha_inicio = datetime(datetime.now().year, 2, 22)

# Obtener el último día del año actual
ultimo_dia_del_ano = datetime(fecha_inicio.year, 12, 31)

# Calcular los días restantes
dias_restantes = (ultimo_dia_del_ano - fecha_inicio).days

# Crear una lista de fechas desde la fecha actual hasta el último día del año
lista_fechas = []

for i in range(dias_restantes + 1):
    f = fecha_inicio + timedelta(days=i)
    g = f.replace(second=0, microsecond=0, minute=0, hour=0)
    
    if i % 6 == 0:  # Realizar la operación cada 4 iteraciones
        lista_fechas.append(g)  # Agregar la fecha a la lista

# Imprimir la lista de fechas
lista_fechas

[datetime.datetime(2024, 2, 22, 0, 0),
 datetime.datetime(2024, 2, 28, 0, 0),
 datetime.datetime(2024, 3, 5, 0, 0),
 datetime.datetime(2024, 3, 11, 0, 0),
 datetime.datetime(2024, 3, 17, 0, 0),
 datetime.datetime(2024, 3, 23, 0, 0),
 datetime.datetime(2024, 3, 29, 0, 0),
 datetime.datetime(2024, 4, 4, 0, 0),
 datetime.datetime(2024, 4, 10, 0, 0),
 datetime.datetime(2024, 4, 16, 0, 0),
 datetime.datetime(2024, 4, 22, 0, 0),
 datetime.datetime(2024, 4, 28, 0, 0),
 datetime.datetime(2024, 5, 4, 0, 0),
 datetime.datetime(2024, 5, 10, 0, 0),
 datetime.datetime(2024, 5, 16, 0, 0),
 datetime.datetime(2024, 5, 22, 0, 0),
 datetime.datetime(2024, 5, 28, 0, 0),
 datetime.datetime(2024, 6, 3, 0, 0),
 datetime.datetime(2024, 6, 9, 0, 0),
 datetime.datetime(2024, 6, 15, 0, 0),
 datetime.datetime(2024, 6, 21, 0, 0),
 datetime.datetime(2024, 6, 27, 0, 0),
 datetime.datetime(2024, 7, 3, 0, 0),
 datetime.datetime(2024, 7, 9, 0, 0),
 datetime.datetime(2024, 7, 15, 0, 0),
 datetime.datetime(2024, 7, 21, 

In [36]:
# Crear un nuevo objeto de calendario
calendario = icalendar.Calendar()

# Crear un evento
evento = icalendar.Event()

# Configurar las propiedades del evento
for day in lista_fechas:
    tempday = day
    for i in range(2):
        evento.add('summary', 'trabajo_dia')
        evento.add('dtstart', tempday.replace(hour=7))
        evento.add('dtend', tempday.replace(hour=19))
        tempday = tempday + timedelta(days=1)
    for i in range(2):
        evento.add('summary', 'trabajo_noche')
        evento.add('dtstart', tempday.replace(hour=19))
        tempday = tempday + timedelta(days=1)
        evento.add('dtend', tempday.replace(hour=7))
    evento.add('summary', 'franco')
    evento.add('dtstart', tempday.replace(hour=7))
    tempday = tempday + timedelta(days=2)
    evento.add('dtend', tempday.replace(hour=7))


# Añadir el evento al calendario
calendario.add_component(evento)

# Guardar el calendario en un archivo .ics
with open('calendario.ics', 'wb') as f:
    f.write(calendario.to_ical())

print("Archivo 'evento.ics' creado con éxito.")

Archivo 'evento.ics' creado con éxito.


In [41]:
# Función para cargar eventos desde un archivo .ics
def cargar_eventos(archivo_ics):
    with open(archivo_ics, 'rb') as f:
        calendario = icalendar.Calendar.from_ical(f.read())
        eventos = []
        for evento in calendario.walk('vevent'):
            inicio = evento.get('dtstart')
            if isinstance(inicio, list):
                inicio = inicio[0]  # Tomar el primer elemento de la lista si es una lista
            inicio = inicio.dt  # Acceder al atributo dt después de obtener el primer elemento
        
            fin = evento.get('dtend')
            if isinstance(fin, list):
                fin = fin[0]  # Tomar el primer elemento de la lista si es una lista
            fin = fin.dt  # Acceder al atributo dt después de obtener el primer elemento

        return eventos

# Función para encontrar el mejor rango de fechas sin eventos
def mejor_rango_sin_eventos(eventos, duracion_minima):
    eventos.sort(key=lambda x: x[0])
    mejor_rango_inicio = datetime.min
    mejor_rango_fin = datetime.min
    maximo_sin_eventos = timedelta()

    for inicio, fin in eventos:
        duracion = inicio - mejor_rango_fin
        if duracion >= duracion_minima:
            if duracion > maximo_sin_eventos:
                mejor_rango_inicio = mejor_rango_fin
                mejor_rango_fin = inicio
                maximo_sin_eventos = duracion

    return mejor_rango_inicio, mejor_rango_fin

# Ejemplo de uso
archivo_ics = 'calendario.ics'  # Cambia esto por la ruta de tu archivo .ics
eventos = cargar_eventos(archivo_ics)
duracion_minima = timedelta(days=1)
mejor_rango_inicio, mejor_rango_fin = mejor_rango_sin_eventos(eventos, duracion_minima)

print("El mejor rango de fechas sin eventos es desde", mejor_rango_inicio, "hasta", mejor_rango_fin)


El mejor rango de fechas sin eventos es desde 0001-01-01 00:00:00 hasta 0001-01-01 00:00:00
